In [5]:
class TooManyMatchesError(Exception):
    pass

class SymbolNotFoundError(Exception):
    pass

class SymbolStore:
    def __init__(self):
        self.symbols = list()
    
    def _get_by(self, **kwargs):
        for s in self.symbols:
            for k, v in kwargs.items():
                if s.get(k, None) != v:
                    break
            else:
                yield s


    def get_first(self, **kwargs):
        try:
            return next(self._get_by(**kwargs))
        except StopIteration as exc:
            raise SymbolNotFoundError("Symbol not found") from exc
    
    def get_one(self, **kwargs):
        i = self._get_by(**kwargs)
        
        try:
            s = next(i)
        except StopIteration as exc:
            raise SymbolNotFoundError("Symbol not found") from exc
        
        try:
            next(i)
        except StopIteration:
            return s
        else:
            raise TooManyMatchesError("More than one match while calling `get_one`")
    
    def get_all(self, **kwargs):
        return list(self._get_by(**kwargs))
    
    def get_or_create(self, **kwargs):
        try:
            s = self.get_one(**kwargs)
        except SymbolNotFoundError:
            s = kwargs.copy()
            self.symbols.append(s)
        return s

In [6]:
symbols = SymbolStore()

In [228]:
import regex
from itertools import groupby
from uuid import uuid4

class Token(str):    
    def __repr__(self):
        return f"{self.__class__.__name__}({super().__repr__()})"
        
class Word(Token):
    pass

class Acronym(Word):
    pass

class Separator(Token):
    pass

class Name:
    _camel_case_regex = regex.compile(r"^([a-z]+)((:?[A-Z0-9][A-Za-z0-9]*?)*)$")
    _full_camel_case_regex = regex.compile(r"^([A-Z][a-z0-9A-Z]*?)((:?[A-Z0-9][A-Za-z0-9]*?)*)$")
    
    def __init__(self, *tokens, **meta):
        self._tokens = list(tokens)
        self._meta = meta

    @classmethod
    def from_snake_case_all_caps(cls, text):
        tokens = []
        for t in text.split('_'):
            tokens.append(Word(t))
            tokens.append(Separator('_'))
            
        return cls(*tokens[:-1], original=text)

    @staticmethod
    def _scan_camel_case_regex(regex, text):
        tokens = []
        if (match := regex.match(text)) is not None:
            captures = [match.group(1)] + list(match.captures(3))
            for _, ws in groupby(captures, key=lambda k: str(k.isalpha()) if len(k) == 1 else repr(uuid4())):
                group = list(ws)
                if len(group) > 1:
                    tokens.append(Acronym(''.join(group)))
                else:
                    tokens.append(Word(group[0]))
            return tokens
        else:
            raise ValueError("Not in camel case format")

    @classmethod
    def from_camel_case(cls, text):
        return cls(*cls._scan_camel_case_regex(cls._camel_case_regex, text), original=text)
    
    @classmethod
    def from_full_camel_case(cls, text):
        return cls(*cls._scan_camel_case_regex(cls._full_camel_case_regex, text), original=text)

    def to_snake_case(self):
        words = []
        for t in self._tokens:
            if isinstance(t, Word):
                words.append(t.lower())
        return '_'.join(words)
    
    def to_snake_case_all_caps(self):
        return self.to_snake_case().upper()
    
    def to_camel_case(self):
        words = []
        is_first_word = True
        for t in self._tokens:
            if isinstance(t, Word):
                if is_first_word:
                    words.append(t.lower())
                    is_first_word = False
                else:
                    words.append(t.capitalize())
        return ''.join(words)        
    
    def to_full_camel_case(self):
        return ''.join(t.capitalize() for t in self._tokens if isinstance(t, Word))

    def __repr__(self):
        return f"Name(*{repr(self._tokens)}, **{repr(self._meta)})"

In [229]:
Name.from_full_camel_case('Ps2PDF')

Name(*[Word('Ps'), Word('2'), Acronym('PDF')], **{'original': 'Ps2PDF'})